In [1]:
import re
import os
import math
import torch
import pandas as pd

from tqdm import tqdm
from rouge import Rouge
from datetime import datetime

from tensorboardX import SummaryWriter
from transformers import AutoTokenizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import Dataset , DataLoader

from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
save_path = os.path.join("./BART_runs", timestamp)

weights_path = os.path.join(save_path, 'weights')
logs_path = os.path.join(save_path, 'logs')
os.makedirs(weights_path, exist_ok=True)
os.makedirs(logs_path, exist_ok=True)

config = {
    "general": {
        "data_path": "../dataset",
        "train_file": "cleaned_train.csv",
        "valid_file": "cleaned_dev.csv",
        "test_file": "test.csv",
        "model_name": "gogamza/kobart-summarization",
        "output_dir": save_path,
        "model_cfg": ""
    },
    "tokenizer": {
        "path": "./tokenizer",
        "encoder_max_len": 512,
        "decoder_max_len": 256,
        "bos_token": "<s>",
        "eos_token": "</s>",
        "sep_token": "<sep>",
        "mask_token": "<mask>",
        "special_tokens": [
            '#Person1#',
            '#Person2#',
            '#Person3#',
            '#Person4#',
            '#Person5#',
            '#Person6#',
            '#Person7#',
            '#SSN#',
            '#Email#',
            '#Address#',
            '#CarNumber#',
            '#Movietitle#',
            '#DateOfBirth#',
            '#CardNumber#',
            '#PhoneNumber#',
            '#PassportNumber#',

            '#Reaction#',
            '#Person8#', 
            '#Person9#', 
            '#Person10#', 
            '#Person11#', 
            '#Manager#',
            '#CompanyName#', 
            '#University#',
            '#PartnerName#', 
            '#Location#',
            '#City#',
            '#Name#', 
            '#CityName#', 
            '#Target#',
            '#Country#',
        ]
    },
    "training": {
        "seed": 42,
        "overwrite_output_dir": True,

        "epochs": 1000,
        "batch_size": 8,
        "learning_rate": 0.000001,
        "weight_decay": 0.01,
        "accumulation_steps": 4,
        "patience": 10,

        "max_lr": 0.0001,
        "warmup_epochs": 10,
        "T_0": 100,
        "T_mult": 1,
        "T_gamma": 0.5,
    },
    "inference": {
        "no_repeat_ngram_size": 2,
        "early_stopping": False,
        "generate_max_length": 200,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": [
            "<unk>",
            "<s>",
            "</s>",
            "<pad>",
            "<sep>",
            "<mask>"
        ]
    }
}


In [ ]:
bart_config = BartConfig.from_pretrained(config['general']['model_name'])

# tokenizer = BartTokenizerFast.from_pretrained("../tokenizer/ko_sentencepiece")
tokenizer = AutoTokenizer.from_pretrained(config['general']['model_name'], config=config)
remove_tokens = ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}", f"{tokenizer.sep_token}", f"{tokenizer.mask_token}"]
unused_tokens = [f"<unused{i}>" for i in range(100)]
tokenizer.add_special_tokens({"additional_special_tokens": config['tokenizer']['special_tokens'] + unused_tokens})
print(tokenizer.special_tokens_map)

model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'])
model.resize_token_embeddings(len(tokenizer))
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config['training']['learning_rate'], weight_decay=config['training']['weight_decay'])

In [4]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, is_train=True):
        self.tokenizer = tokenizer
        self.df = df.copy()
        self.is_train = is_train

        # 화자가 바뀔 때 SEP 토큰을 추가
        self.df.loc[:, 'dialogue'] = self.df['dialogue'].apply(self.add_sep_tokens)

        if self.is_train:           
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), 
                                       return_tensors="pt", 
                                       padding=True,
                                       add_special_tokens=True, 
                                       truncation=True, 
                                       max_length=config['tokenizer']['encoder_max_len'], 
                                       return_token_type_ids=False).input_ids
            
            self.labels = tokenizer(self.df['summary'].tolist(), 
                                    return_tensors="pt", 
                                    padding=True,
                                    add_special_tokens=True, 
                                    truncation=True, 
                                    max_length=config['tokenizer']['decoder_max_len'],
                                    return_token_type_ids=False).input_ids
        else:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), 
                                       return_tensors="pt", 
                                       padding=True,
                                       add_special_tokens=True, 
                                       truncation=True, 
                                       max_length=config['tokenizer']['encoder_max_len'], 
                                       return_token_type_ids=False).input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.is_train:
            return self.input_ids[idx], self.labels[idx]
        else:
            return self.input_ids[idx]

    def add_sep_tokens(self, dialogue):
        # 화자가 바뀔 때 SEP 토큰을 추가
        pattern = r'(#Person\d+#)'  # 화자를 나타내는 패턴
        parts = re.split(pattern, dialogue)  # 화자를 기준으로 대화 분리
        result = []
        prev_speaker = None
        for part in parts:
            if re.match(pattern, part):  # 화자가 바뀌면
                if prev_speaker and prev_speaker != part:
                    result.append('<sep>')  # SEP 토큰 추가
                prev_speaker = part
            result.append(part)
        return ''.join(result)

In [ ]:
train_df = pd.read_csv(f"{config['general']['data_path']}/{config['general']['train_file']}")
val_df = pd.read_csv(f"{config['general']['data_path']}/{config['general']['valid_file']}")
test_df = pd.read_csv(f"{config['general']['data_path']}/{config['general']['test_file']}")

gen_df = pd.read_csv("../dataset/generated_train.csv")
gen_df = gen_df.dropna(subset=['dialogue', 'summary'])

trans_train = pd.read_csv("../dataset/processed_translated_train.csv")

train_df = pd.concat([train_df, gen_df, trans_train])
print(train_df.shape, val_df.shape, test_df.shape)

train_dataset = CustomDataset(train_df[['dialogue', 'summary']], tokenizer)
val_dataset = CustomDataset(val_df[['dialogue', 'summary']], tokenizer)
test_dataset = CustomDataset(test_df[['dialogue']], tokenizer, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['training']['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config['training']['batch_size'], shuffle=False)

In [ ]:
batch = next(iter(train_loader))
masked_input_ids, original_input_ids = batch[0], batch[1]

masked_example = tokenizer.decode(masked_input_ids[0], skip_special_tokens=False)
original_example = tokenizer.decode(original_input_ids[0], skip_special_tokens=False)

masked_example, original_example

In [7]:
def ids_to_words(tokenizer, preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()

    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]
        
    return replaced_predictions, replaced_labels

In [8]:
def compute_metrics(replaced_predictions, replaced_labels):
    rouge = Rouge()

    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    result = {key: value["f"] for key, value in results.items()}
    
    return result

In [9]:
class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super().__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [10]:
def train(epoch, model, device, train_loader, optimizer, writer, accumulation_steps):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training Epoch {epoch}", leave=False):
        input_ids = batch[0].to(device, dtype=torch.long)
        labels = batch[1].to(device, dtype=torch.long)

        outputs = model(input_ids=input_ids, labels=labels)
        ce_loss = outputs.loss
        ce_loss = ce_loss / accumulation_steps 
        ce_loss.backward()

        if (idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += ce_loss.item() * accumulation_steps

    avg_loss = total_loss / len(train_loader)
    writer.add_scalar('Loss/train', avg_loss, epoch)

    return avg_loss

def validate(tokenizer, model, device, val_loader, writer, epoch):
    model.eval()
    total_loss = 0
    all_results = []
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for idx, batch in tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating", leave=False):
            input_ids = batch[0].to(device, dtype=torch.long)
            labels = batch[1].to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=config['inference']['generate_max_length'], 
                num_beams=config['inference']['num_beams'],
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )

            loss = model(input_ids=input_ids, labels=labels).loss
            total_loss += loss.item()

            replaced_predictions, replaced_labels = ids_to_words(tokenizer, pred_ids, labels)
            result = compute_metrics(replaced_predictions, replaced_labels)

            all_results.append(result)
            all_predictions.extend(replaced_predictions)
            all_labels.extend(replaced_labels)

    val_loss = total_loss / len(val_loader)
    avg_result = {key: sum(r[key] for r in all_results) / len(all_results) for key in all_results[0]}
    
    writer.add_scalar('Loss/valid', val_loss, epoch)
    writer.add_scalar('ROUGE/rouge-1', avg_result['rouge-1'], epoch)
    writer.add_scalar('ROUGE/rouge-2', avg_result['rouge-2'], epoch)
    writer.add_scalar('ROUGE/rouge-l', avg_result['rouge-l'], epoch)

    return val_loss, avg_result, all_predictions, all_labels

In [ ]:
best_val_loss = float('inf')
early_stopping_counter = 0
writer = SummaryWriter(log_dir=logs_path)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=config['training']['T_0'], T_mult=config['training']['T_mult'], eta_max=config['training']['max_lr'],  T_up=config['training']['warmup_epochs'], gamma=config['training']['T_gamma'])

for epoch in range(1, config['training']['epochs'] + 1):
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch}/{config['training']['epochs']}, Current Learning Rate: {current_lr:.6f}")
    
    train_loss = train(epoch, model, device, train_loader, optimizer, writer, config['training']['accumulation_steps'])    
    val_loss, val_result, val_predictions, val_labels = validate(tokenizer, model, device, val_loader, writer, epoch)

    print(f"Train Loss: {train_loss:.6f}, Valid Loss: {val_loss:.6f}")
    print(f"Rouge-1: {val_result['rouge-1']:.6f}, Rouge-2: {val_result['rouge-2']:.6f}, Rouge-l: {val_result['rouge-l']:.6f}")

    scheduler.step()
    print('-'*150)
    
    for i in range(3):
        print(f"PRED: {val_predictions[i].strip()}")
        print(f"GOLD: {val_labels[i]}")
        print('-'*150)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), os.path.join(weights_path, 'best_finetune.pth'))
        print(f"New best model saved with validation loss: {best_val_loss:.6f}")
    
    else:
        early_stopping_counter += 1
        print(f"Not improved. Early stopping counter: {early_stopping_counter}/{config['training']['patience']}")

    if early_stopping_counter >= config['training']['patience']:
        print("Early stopping triggered.")
        break

    print()

writer.close()
torch.save(model.state_dict(), os.path.join(weights_path, 'last_finetune.pth'))
print("Training completed. Last model saved.")


In [12]:
def predict(tokenizer, model, device, test_loader, fname):
    model.eval()
    summary = []
    with torch.no_grad():
        for input_ids in tqdm(test_loader):
            input_ids = input_ids.to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=config['inference']['generate_max_length'], 
                num_beams=config['inference']['num_beams'],
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )
            for ids in pred_ids:
                result = tokenizer.decode(ids)
                summary.append(result)
                
    # remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": fname,
            "summary" : preprocessed_summary,
        }
    )
    return output

In [ ]:
model.load_state_dict(torch.load(f'{save_path}/weights/best_finetune.pth'))
output = predict(tokenizer, model, device, test_loader, test_df['fname'])
output.to_csv(f"{save_path}/prediction.csv", index=False)